# Grid Problem Introduction

The premise of this problem is a simple one - we create a deterministic MDP with `n` distinct states and `m` distinct actions, with each state having a (non-unique) reward. We want to see if the splitting algorithm can learn the states based simply on the initial reward-based clustering and by observing contradictions in transitions. 

## Dataset Creation

First, we load the relevant functions - be sure to append the relevant directory with the MDP algorithm by using `sys.path.append`! 

In [1]:
# Importing libararies and loading functions
import numpy as np
import matplotlib.pyplot as plt
import random

# Initializing file-path
import sys
sys.path.append('/Users/janiceyang/Dropbox (MIT)/ORC UROP/Opioids/Algorithm/')

from model import MDP_model
from MDPtools import Generate_random_MDP, sample_MDP_with_features_list
from clustering import initializeClusters, splitter, split_train_test_by_id
from testing import *
from grid_functions import * 

Now, we are going to create the a random MDP and set the rewards for each state of the MDP. `n` is the number of states, `m` is the number of actions, and we can also specify if we want a deterministic or action-dependent reward. Finally, we arbitrarily set some overlapping reward values for different clusters just to see how good our algorithm is at distinguishing between the clusters even though they will intially look the same (based on reward-based clustering).

In [2]:
# Defining parameters
n = 15
m = 3
reward_dep_action = False
deterministic = True

# Generating the actual MDP
P, R = Generate_random_MDP(n,
                           m,
                           reward_dep_action=reward_dep_action,
                           deterministic=deterministic)

# Altering some of the rewards so that there is some overlap
for i in range(n):
    R[i] = i%6*0.2

Now that we have our MDP, we can start generating some sample data by simulating some paths through the system. Here, `N` is the number of actors we want to simulate, and `T` is the total number of timesteps we want each to take. We will generate these samples by using a normal distribution with a `sigma` amount of noise, so that visually, each state will appear to take up a specific portion of an x-y grid (you will see below!). 

In [3]:
# Defining parameters
pfeatures = 2
sigma = [[0.01, 0], [0, 0.01]]
N = 250
T = 5

# Generating the normal distribution based on sigma noise
normal_distributions = UnifNormal(n,
                                     pfeatures,
                                     sigma)

# Generating a list of samples based on the distribution 
samples = sample_MDP_with_features_list(P,
                                        R,
                                        normal_distributions,
                                        N,
                                        T)

Finally, we transform this list of samples into a dataframe, which is what we will feed into the algorithm! The column `OG_CLUSTER` corresponds to the state that this point was in in the MDP we generated earlier. We will use compare this with the `CLUSTER` our algorithm finds to see how well it performs.

In [4]:
df = transformSamples(samples,
                      pfeatures)
df

ID  TIME  FEATURE_1  FEATURE_2  ACTION  RISK  OG_CLUSTER
0       0     0  -0.078803   2.115858       1   0.4           2
1       0     1   2.005336   0.032471       2   0.4           8
2       0     2   0.145247   2.835660       1   0.6           3
3       0     3   3.022813   0.997769       2   0.2          13
4       0     4  -0.060527   2.002646       0   0.4           2
...   ...   ...        ...        ...     ...   ...         ...
1245  249     0  -0.091379   0.886343       0   0.2           1
1246  249     1   2.061035   0.994344       1   0.6           9
1247  249     2   2.859373   0.191191       2   0.0          12
1248  249     3  -0.049313   1.997870       1   0.4           2
1249  249     4   2.052175  -0.056084       1   0.4           8

[1250 rows x 7 columns]